In [1]:
# Dependencies
from splinter import Browser
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import time

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [3]:
# URL of page to be scraped
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(url)

In [4]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [5]:
#Scrape the NASA Mars News Site and collect the latest News Title
news_title = soup.find('div', class_="image_and_description_container").h3.text
news_title

"NASA's Perseverance Rover Mission Getting in Shape for Launch"

In [6]:
#Scrape the NASA Mars News Site and collect the latest News Paragraph Text
news_p=soup.find('div', class_="article_teaser_body").text
news_p

'Stacking spacecraft components on top of each other is one of the final assembly steps before a mission launches to the Red Planet. '

In [7]:
#Use splinter to navigate the site and find the image url for the current Featured Mars Image
url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url)

In [8]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [9]:
browser.click_link_by_partial_text('FULL IMAGE')

C:\Users\chand\Anaconda3\lib\site-packages\splinter\driver\webdriver\__init__.py:528: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


In [10]:
#scrape featured image
url = 'https://www.jpl.nasa.gov'+soup.find('a', class_="button fancybox").get("data-link")
browser.visit(url)

In [11]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [12]:
featured_image_url = "https://www.jpl.nasa.gov" +soup.find('figure', class_='lede').a.img.get('src')
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA17474_hires.jpg'

In [13]:
url = "https://twitter.com/marswxreport?lang=en"
browser.visit(url)
time.sleep(1)

In [14]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [15]:
mars_weather = soup.body.find('div',{"class":["css-901oao","r-hkyrab","r-1qd0xha","r-a023e6","r-16dba41","r-ad9z0x","r-bcqeeo","r-bnwqim","r-qvutc0"]}, lang='en').find('span', {"class":["css-901oao","css-16my406","r-1qd0xha","r-ad9z0x","r-bcqeeo","r-qvutc0"]}).text
mars_weather

'InSight sol 517 (2020-05-10) low -92.6ºC (-134.7ºF) high 1.2ºC (34.2ºF)\nwinds from the SW at 4.9 m/s (11.0 mph) gusting to 17.5 m/s (39.2 mph)\npressure at 7.00 hPa'

In [16]:
url = "https://space-facts.com/mars/"
browser.visit(url)
time.sleep(1)

In [21]:
html = browser.html
mars_facts_df = pd.read_html(html)

In [22]:
mars_facts_df = mars_facts_df[0].rename(columns={0:'Description',1:'Value'})

mars_facts_df.to_html("Mars_Facts.html")

In [24]:
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url)
time.sleep(1)

In [25]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [26]:
title_list = []
img_url_list = []

for name in soup.body.find_all('h3'):
    title_list.append(name.text)
    browser.click_link_by_partial_text(name.text[0:6])
    time.sleep(1)
    browser.click_link_by_partial_text('Sample')
      
    browser.windows[1].is_current=True
    
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    img_url_list.append(soup.img.get('src'))
    
    browser.windows[1].close()
    browser.back()
    
    time.sleep(1)

In [27]:
hemisphere_image_urls =[]
for i in range(0,4):
    myDict = {"title":title_list[i],"img_url":img_url_list[i]}
    hemisphere_image_urls.append(myDict)
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [30]:
browser.windows[0].close()

WebDriverException: Message: no such session
  (Driver info: chromedriver=2.34.522940 (1a76f96f66e3ca7b8e57d503b4dd3bccfba87af1),platform=Windows NT 10.0.18362 x86_64)
